# Discovery dos dados

Neste notebook meu objetivo é fazer um check de sanidade das informações. Eu pesquisei e o dataset é bem visto na comunidade. 
Meu objetivo aqui, até pelo tempo que tenho disponível para esse projeto, não vai ser me aprofundar e validar todas as informações aqui presentes, mas neste notebook eu quero dar uma olhada geral pra ver se o que tem aqui faz sentido. E o que não fizer sentido eu vou tentar entender um pouco melhor.

In [ ]:
# Preciso fazer a config abaixo para que os imports funcionem corretamente

import os
from pathlib import Path
import utils as fxns

# Caminho absoluto da raiz do projeto
project_root = Path.cwd().parent  # se o notebook está em /notebooks
os.chdir(project_root)

print("Diretório atual:", Path.cwd())

In [ ]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
from src.modules.db_reader import DbReader
from src.data_viz.plotter import Plotter
f1_db = DbReader()
plotter = Plotter()

# Olhada geral nos dados de qualificação:

In [ ]:
df_quali = f1_db.run_query_file("data/db_queries/qualify_report.sql").drop_duplicates()

In [ ]:
df_quali

In [ ]:
df_poles = df_quali[df_quali["position"] == 1].copy()

In [ ]:
df_poles

In [ ]:
df_count_pole_por_piloto = df_poles[
    df_poles['session_type'] == 'Q3'
].groupby("driver_surname").size().reset_index(name="pole_count")

In [ ]:
df_count_pole_por_piloto.sort_values(by='pole_count', ascending=False)

In [ ]:
plotter.barplot(
    df=df_count_pole_por_piloto.sort_values("pole_count", ascending=False).head(20),
    x="driver_surname",
    y="pole_count",
    palette="viridis",
    figsize=(12, 8),
    title="Top 20 Pilotos por Pole Positions"
)

Vendo o gráfico acima, parece que o dataset não é 100% perfeito, o Schumacher teria mais poles do que o indicado, e o Hamilton por exemplo não tem 107, e sim 104.

## Vou tentar analisar a integralidade dos dados, e o quão errado pode estar:

### Primeiro vou ver a quantidade de dados por ano:

In [ ]:
df_quali

In [ ]:
df_eventos_quali_por_ano = df_quali[["year", "race_name"]].drop_duplicates().sort_values("year")

In [ ]:
plotter.barplot(
    df=df_eventos_quali_por_ano,
    x="year",              # eixo X
    groupby="year",         # agrupa por ano
    count_name="count",     # nome da contagem
    palette="viridis",
    figsize=(12, 8),
    title="Eventos de Qualificação por Ano"
)


Olhando o gráfico acima e dando uma pesquisada superficial, parece que os dados estão completos a partir de 2003.

### Tentando entender porque o Hamilton tem pole a mais do que deveria:

In [ ]:
df_quali_ham = df_quali[(df_quali["driver_surname"] == "Hamilton") & (df_quali['session_type']=='Q3')].copy()

In [ ]:
df_quali_ham

In [ ]:
df_poles_ham = df_quali_ham[df_quali_ham["position"] == 1].copy()

In [ ]:
df_poles_ham

In [ ]:
plotter.barplot(
    df=df_poles_ham,
    x="year",                   # eixo X
    groupby="year",              # agrupa por ano
    count_name="pole_count",     # nome da contagem
    palette="viridis",           # paleta de cores
    figsize=(12, 8),
    title="Poles do Hamilton por Ano"
)


Por exemplo, aqui fala que o Hamilton tem 5 poles em 2007, mas em tese deveria ser 6. E o que parece que tá faltando é a pole dele na Hungria.

In [ ]:
df_quali[(df_quali["year"] == 2007) & (df_quali["race_name"] == "Hungarian Grand Prix")]

Analisando os dados mais a fundo e pesquisando sobre o GP da Hungria: o Hamilton oficialmente fez a pole, porém o que aconteceu foi que o na verdade quem fez o melhor tempo foi o Alonso (conforme os dados mostram corretamente), mas o Alonso foi penalizado e não largou em primeiro na corrida. Então o dataset está correto no sentido de mostrar quem fez os melhores tempos mas não necessariamente vai mostrar quem vai largar em primeiro de fato caso tenha acontecido alguma penalização. Isso potencialmente pode ser corrigido pelo dataset de corridas, que tem a posição de largada de cada piloto.

Ao mesmo tempo, em 2012 ele teria feito 7 poles e não 8, vamos entender:

In [ ]:
df_quali_ham_2012 = df_quali_ham[df_quali_ham["year"] == 2012].copy()

In [ ]:
df_quali_ham_2012

Analisando os dados e dando uma pesquisada a respeito chego em conclusões parecidas com o que vi acima sobre o GP das Hungria de 2007: em 2012 no GP da Espanha o Hamilton fez a volta mais rápida (como os dados mostram), mas na verdade ele largou de último porque o carro dele estava irregular. Então em termos de analisar os tempos, parece que esse dataset vai estar correto para os dados que estão presentes mas não vai respeitar eventuais desclassificações etc sendo que imagino que isso vai ser respeitado no dataset de corridas (o que faz sentido). Em breve, vou dar uma olhada no dataset de corridas pra confirmar. 

# Dados de Corrida:

In [ ]:
df_race = f1_db.run_query_file("data/db_queries/race_results_report.sql").drop_duplicates()

In [ ]:
df_race

In [ ]:
# Vou comparar a as quantidades de poles em relação ao que vimos no dataset de qualificação pra comparar:

plotter.barplot(
    df=df_race,
    x="driver_full_name",                  # eixo X
    filter_query="starting_position == 1", # filtro
    groupby="driver_full_name",            # agrupamento
    count_name="pole_count",               # nome da coluna de contagem
    top_n=20,                              # top 20
    sort_by="pole_count",                  # ordena pela contagem
    ascending=False,                       # do maior pro menor
    palette="viridis",                     # paleta de cores
    figsize=(12, 8),                        # tamanho da figura
    title="Top 20 Pilotos em Largadas na Pole"
)


In [ ]:
# A maioria parece certo, mas o Jim Clark tem 1 pole a mais no dataset de corrrida do que ele deveria, vou tentar investigar isso:

df_race_clark = df_race[df_race['driver_full_name'] == "Jim Clark"].copy()

In [ ]:
df_race_clark[df_race_clark['starting_position'] == 1].groupby("year").size().reset_index(name="pole_count").plot(kind='bar', x='year', y='pole_count', figsize=(12, 8), title="Poles de Jim Clark por Ano")

In [ ]:
# O problema está em 1964

df_race_clark[(df_race_clark['year'] == 1964) & (df_race_clark['starting_position'] == 1)].copy()

O Jim Clark, por incrível que pareça, realmente "terminou" essa corrida 2 vezes. O carro dele tava pra quebrar e aí trocaram ele de carro, então ele saiu de um carro e depois o outro quebrou também (risos). De resto, parece até que tá ok. Agora vamos dar uma olhada em vitórias:

In [ ]:
plotter.barplot(
    df_race,
    x="driver_full_name",
    filter_query="finishing_position == 1",
    groupby="driver_full_name",
    count_name="win_count",
    top_n=20,
    sort_by="win_count",
    title="Top 20 Pilotos por Vitórias",
)

Esse dataset está parecendo fazer sentido em relação às quantidades.

Vi no Kaggle um cara falando que tem um erro específico no GP do Japão de 2019 em relação a quem ganhou a corrida. Vamos dar uma olhada:

In [ ]:
df_race_jp_2019 = df_race[(df_race['year'] == 2019) & (df_race['race_name'] == "Japanese Grand Prix")].copy()

In [ ]:
df_race_jp_2019

Devem ter arrumado o dataset, pq parece que o vencedor está correto, realmente foi o Bottas que ganhou essa corrida.

É difícil continuar a validação pq não vou ficar olhando o dataset inteiro com todos os dados em detalhe, mas o que eu vi superficialmente desse dataset de resultados de corrida, eu estou contente e de boa para seguir com ele por ora.

# Dados de tempos de voltas:

In [ ]:
df_lap_times = f1_db.run_query_file("data/db_queries/lap_times_report.sql").drop_duplicates()

In [ ]:
df_lap_times

### Tentando ver o quão completo está o dataset:

In [ ]:
# Posso ver a quantidade de eventos por ano, pra ter uma ideia geral do quanto e a partir de quando temos mais completude de dados:

df_lap_times_eventos_anos = df_lap_times[['year', 'race_name']].drop_duplicates().sort_values("year")
df_lap_times_eventos_anos

In [ ]:
plotter.barplot(
    df_lap_times_eventos_anos,
    x="year",
    groupby="year",
    count_name="Event Count",
    sort_by="year",
)

Dando uma pesquisada geral, parece que tá tudo aqui a partir de 1996.

In [ ]:
df_lap_times.sort_values("lap_time", ascending=False)

In [ ]:
# Vou olhar o maior tempo de todos pra ver se ele tá faznedo sentido, que é do Massa na Hungria em 2005

df_massa_hungria = fxns.filtrar_evento(
    df_lap_times,
    year=2005,
    circuit_name="Hungaroring",
    driver_full_name="Felipe Massa",
)

In [ ]:
df_massa_hungria

Realmente, parece que o Massa teve um problema nessa corrida e ele parou nos boxes na volta 42 pra tentar resovler (no Wikipedia fala que ele passou várias voltas no Box?). Então possivelmente este dado está correto, mas me questiono se faz sentido usá-lo em análises para comparar pilotos etc. Sendo que esses tempos de volta serão sempre totais outliers, quando o piloto teve algum problema.

Mas, para efeito de validação dos dados, vou considerar que o que está aqui, está ok!

### Vou dar uma olhada nas voltas de pit stop pra garantir que estão sendo classificadas de forma coerente:

In [ ]:
df_lap_times

# Vou criar uma coluna com o tempo de volta em ms totais, pq aí fica mais fácil de eu ordenar e fazer outras contas depois

df_lap_times = fxns.add_lap_time_ms_column(df_lap_times)

Espero que cada piloto tenha pelo menos 1 pit stop por prova e que a volta seja bem mais lenta:

In [ ]:
df_check_ps = df_lap_times.groupby(by=['race_name', 'race_date', 'driver_full_name', 'is_pit_lap']).agg(
    {
        'lap_number':'count',
        'lap_time_ms':'mean'
    }
).reset_index().rename(columns={'lap_number': 'lap_count'})

df_check_ps

Interessante que parece que o Carlos Sainz e outros pilotos tiveram 4 paradas no GP de Aniversário de 70 anos da F1, vou dar uma olhada se é isso mesmo:

In [ ]:
fxns.filtrar_evento(
    df=df_lap_times,
    race_name='70th Anniversary Grand Prix',
    driver_full_name='Carlos Sainz'
)

Está correto, o que acontece é que, como classificamos tanto a in-lap quanto a out-lap (tanto a volta que o piloto entra quanto a que ele sai dos boxes) como *"is_pit_lap"*, então o número de *pit laps* sempre vai ser o dobro da quantidade de paradas e ele realmente parou 2 vezes nessa corrida. Ou seja, pelo menos em uma análise pontual, parece que a flag está correta.

In [ ]:
df_check_ps.groupby('is_pit_lap').agg({'lap_count':'mean', 'lap_time_ms': 'mean'})

Em média, o número de pit laps é de quase 4, o que significa 2 paradas em média por corrida e o tempo de volta quando a flag é 1 é bem maior do que quando é 0, o que faz sentido.